In [5]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd

from scipy.io import wavfile
import noisereduce as nr
%xmode Verbose

Exception reporting mode: Verbose


In [6]:
noisy_speech = 'audio_test_files/430_AUDIO_SLICED_30.0s.wav'
my_voice = 'audio_test_files/satz mit pause.wav'
FRAME_SIZE = 662
HOP_LENGTH = 166

In [7]:
ipd.Audio(noisy_speech)

In [8]:
ipd.Audio(my_voice)

In [9]:
signal, _ = librosa.load(noisy_speech, offset=-5.0, duration=5.0)
my_voice_signal, sr = librosa.load(my_voice)
print(len(signal))

C:\Users\oliver\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


ValueError: Input signal length=0 is too small to resample from 16000->22050

In [10]:
%debug

> c:\users\oliver\anaconda3\lib\site-packages\resampy\core.py(97)resample()
     95 
     96     if shape[axis] < 1:
---> 97         raise ValueError('Input signal length={} is too small to '
     98                          'resample from {}->{}'.format(x.shape[axis], sr_orig, sr_new))
     99 

ipdb> up
> c:\users\oliver\anaconda3\lib\site-packages\librosa\core\audio.py(604)resample()
    602         y_hat = soxr.resample(y.T, orig_sr, target_sr, quality=res_type).T
    603     else:
--> 604         y_hat = resampy.resample(y, orig_sr, target_sr, filter=res_type, axis=-1)
    605 
    606     if fix:

ipdb> up
> c:\users\oliver\anaconda3\lib\site-packages\librosa\core\audio.py(175)load()
    173 
    174     if sr is not None:
--> 175         y = resample(y, sr_native, sr, res_type=res_type)
    176 
    177     else:

ipdb> up
> <ipython-input-9-bfa7e7322632>(1)<module>()
----> 1 signal, _ = librosa.load(noisy_speech, offset=-5.0, duration=5.0)
      2 my_voice_signal, sr = librosa.

In [ ]:
zcr_signal = librosa.feature.zero_crossing_rate(signal, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
zcr_my_voice = librosa.feature.zero_crossing_rate(my_voice_signal, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]

In [ ]:
frames_signal = range(len(zcr_signal))
t_signal = librosa.frames_to_time(frames_signal, hop_length=HOP_LENGTH)
frames_my_voice = range(len(zcr_my_voice))
t_my_voice = librosa.frames_to_time(frames_my_voice, hop_length=HOP_LENGTH)

In [ ]:
plt.figure(figsize=(30, 17)) #width & height for figure - figure holds diferent subplots
plt.subplots_adjust(wspace=0.5,hspace=0.5)
plt.subplot(2, 1, 1)
librosa.display.waveplot(signal, alpha=0.5) # alpha is transparency
plt.plot(t_signal, zcr_signal, color="r")
plt.ylabel('Amplitude', fontsize=20)
plt.xlabel('Zeit (s)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("noisy speech signal - ('How are you doing today? Good!')", fontsize=20)

plt.subplot(2, 1, 2)
librosa.display.waveplot(my_voice_signal, alpha=0.5) # alpha is transparency
plt.plot(t_my_voice, zcr_my_voice, color="r")
plt.ylabel('Amplitude', fontsize=20)
plt.xlabel('Zeit (s)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Sprachaufnahme - ('Dies ist ein Satz... mit Pause.')\n Zeitsignal (blau) + ZCR (rot)", fontsize=20)

# Noise reduction

In [ ]:
# load data
rate, data = wavfile.read(noisy_speech)
# perform noise reduction
reduced_noise = nr.reduce_noise(y=data, sr=rate, n_fft=512)
wavfile.write("mywav_reduced_noise.wav", rate, reduced_noise)

In [ ]:
denoised, _ = librosa.load("audio_test_files/mywav_reduced_noise.wav")

In [ ]:
zcr_denoised = librosa.feature.zero_crossing_rate(denoised, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
print(zcr_denoised.shape)

In [ ]:
frames_denoised = range(len(zcr_denoised))
t_denoised = librosa.frames_to_time(frames_denoised, hop_length=HOP_LENGTH)

In [ ]:
plt.figure(figsize=(30, 17)) #width & height for figure - figure holds diferent subplots
plt.subplot(2, 1, 1)
librosa.display.waveplot(denoised, alpha=0.5) # alpha is transparency
#plt.plot(t_denoised, zcr_denoised, color="r")
plt.ylabel('Amplitude', fontsize=20)
plt.xlabel('Zeit (s)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Denoised speech signal", fontsize=20)
# kaum amplitude bei rauschen aber hoher ausschlag bei zcr in best. beriechen